In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()


In [ ]:
movies = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/online") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "movies") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

reviews = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/online") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "reviews") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [ ]:
movies.show()

In [ ]:
reviews.show()

In [ ]:
from pyspark.sql.functions import avg,ceil,repeat,lit
avg_df = movies\
    .join(reviews, movies.id == reviews.movie_id, "inner")\
    .groupBy(col("genre"),col("title"))\
    .agg(ceil(avg("rating")).alias("avg_rating"))

avg_df = avg_df\
    .withColumn("stars_string", lit("*"))\
    .withColumn("avg_rating_str", avg_df["avg_rating"].cast("string"))\
    .withColumn("stars", repeat(col("stars_string"), col("avg_rating_str").cast("int")))\
    .drop("avg_rating_str") 